In [86]:
import pandas as pd 
import matplotlib.pyplot as plt

In [87]:
import sys
import os

# Add the directory containing the module to the system path
module_path = os.path.abspath(os.path.join('C:\\Users\\samue\\OneDrive\\AIML\\HS2024\\Data Sicence Projekt\\HSLU_DSPRO1_TrafficStatus\\data'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import the module
import data_engineering_library as dlib

In [88]:
path_from = r"C:\Users\samue\OneDrive\AIML\HS2024\Data Sicence Projekt\Data\London\London_UTD19.csv"
path_detectors = r"C:\Users\samue\OneDrive\AIML\HS2024\Data Sicence Projekt\Data\London\London_detectors.csv"

print("Loading data from: ", path_from)
dataframe_London_UTD19 = pd.DataFrame(dlib.load_data(path=path_from, nrows=1000000))
print("Loading data from: ", path_detectors)
dataframe_detectors = pd.DataFrame(dlib.load_data(path=path_detectors))
print("Data loaded")

Loading data from:  C:\Users\samue\OneDrive\AIML\HS2024\Data Sicence Projekt\Data\London\London_UTD19.csv
Loading data from:  C:\Users\samue\OneDrive\AIML\HS2024\Data Sicence Projekt\Data\London\London_detectors.csv
Data loaded


In [89]:
dataframe_London_UTD19, errors = dlib.preprocess_dataframe(dataframe_London_UTD19)
print(f"Errors found and dropped: {errors}")

Errors found and dropped: 336130


In [90]:
dataframe_London_UTD19 = dlib.drop_false_values_by_date(dataframe_London_UTD19, column='flow')

Total outliers detected and removed: 0


In [91]:
dataframe_London_UTD19 = dlib.clip_outliers(dataframe_London_UTD19, column='occ', group_by_detid=True, outlier_factor=3)

Total outliers clipped: 113


In [92]:
dataframe_London_UTD19 = dlib.clip_outliers(dataframe_London_UTD19, column='flow', group_by_detid=True, outlier_factor=3)

Total outliers clipped: 2


In [ ]:
dataframe_London_UTD19 = dlib.calculate_traffic_speed(dataframe_London_UTD19)§

In [94]:
dataframe_London_UTD19 = dlib.drop_outliers(dataframe_London_UTD19, column='traffic', group_by_detid=True, outlier_factor=2)

Total outliers dropped: 15668


In [95]:
dataframe_London_UTD19 = dlib.drop_false_values_by_date(dataframe_London_UTD19, column='traffic')

Total outliers detected and removed: 0


In [96]:
dataframe_anomalies = dlib.detect_anomalies(dataframe_London_UTD19, column='traffic', factor=3, min_IQR=5, min_data_points=4000)

Anomalies detected based on IQR: 2
Anomalies detected based on IQR too small: 4
Anomalies detected based on not enough data: 42


In [97]:
dataframe_London_UTD19, dataframe_anomalies = dlib.handle_anomalies(dataframe_London_UTD19, dataframe_anomalies)

Anomalies with not enough data handled: 15
Total amount of dropeed anomalies: 29


In [98]:
min_traffic = dataframe_London_UTD19['traffic'].min()
mean_traffic = dataframe_London_UTD19['traffic'].mean()
max_traffic = dataframe_London_UTD19['traffic'].max()
print(f"Minimum traffic: {min_traffic}")
print(f"Mean traffic: {mean_traffic}")
print(f"Maximum traffic: {max_traffic}")

Minimum traffic: 0.0
Mean traffic: 79.42519487178676
Maximum traffic: 821.4999735


In [99]:
dataframe_London_UTD19 = dlib.combine_datapoints(dataframe_London_UTD19, ratio=3600)

In [100]:
min_traffic = dataframe_London_UTD19['traffic'].min()
mean_traffic = dataframe_London_UTD19['traffic'].mean()
max_traffic = dataframe_London_UTD19['traffic'].max()
print(f"Minimum traffic: {min_traffic}")
print(f"Mean traffic: {mean_traffic}")
print(f"Maximum traffic: {max_traffic}")

Minimum traffic: 0.0
Mean traffic: 78.56649290647242
Maximum traffic: 660.48


In [101]:
zero_values = dataframe_London_UTD19[dataframe_London_UTD19['traffic'] <= 1]
zero_values.count()

interval    2342
day         2342
detid       2342
weekday     2342
flow        2342
occ         2342
traffic     2342
dtype: int64

In [102]:
dataframe_London_UTD19 = dlib.normalize_traffic(dataframe_London_UTD19)

In [103]:
min_traffic = dataframe_London_UTD19['traffic'].min()
mean_traffic = dataframe_London_UTD19['traffic'].mean()
max_traffic = dataframe_London_UTD19['traffic'].max()
print(f"Minimum traffic: {min_traffic}")
print(f"Mean traffic: {mean_traffic}")
print(f"Maximum traffic: {max_traffic}")

Minimum traffic: 0.0
Mean traffic: 11.776409274680189
Maximum traffic: 99.0


In [104]:
dataframe_London_UTD19 = dlib.merge_dataframes_on_detid(dataframe_London_UTD19, dataframe_detectors)

In [105]:
zero_values = dataframe_London_UTD19[dataframe_London_UTD19['traffic'] <= 1]
zero_values.count()

interval    8256
day         8256
detid       8256
weekday     8256
flow        8256
occ         8256
traffic     8256
lanes       8256
long        8256
lat         8256
pos         8256
length      8256
dtype: int64

In [106]:
dataframe_London_UTD19 = dlib.final_process_dataframe(dataframe_London_UTD19)
zero_values = dataframe_London_UTD19[dataframe_London_UTD19['traffic'] == 0]
zero_values.count()

interval    8256
day         8256
detid       8256
weekday     8256
traffic     8256
long        8256
lat         8256
pos         8256
length      8256
dtype: int64

In [107]:
na_counts = dataframe_London_UTD19.isna().sum()
print(na_counts)

interval    0
day         0
detid       0
weekday     0
traffic     0
long        0
lat         0
pos         0
length      0
dtype: int64


In [ ]:
path_dataset = r"C:\Users\samue\OneDrive\AIML\HS2024\Data Sicence Projekt\Data\London_UTD19_Modified.csv"
dataframe_London_UTD19_Modified = dlib.load_data(path=path_dataset, nrows=None)

In [ ]:
min_traffic = dataframe_London_UTD19_Modified['traffic'].min()
mean_traffic = dataframe_London_UTD19_Modified['traffic'].mean()
max_traffic = dataframe_London_UTD19_Modified['traffic'].max()
print(f"Minimum traffic: {min_traffic}")
print(f"Mean traffic: {mean_traffic}")
print(f"Maximum traffic: {max_traffic}")